In [2]:
import sys
import pandas as pd
from orderbook import OrderBook
import glob

In [ ]:
df = pd.read_parquet('CL'/'outright'/'CLF5'/'20240101'/)

In [ ]:
def rebuild_orderbook(parquet_path: str) -> OrderBook:
    # Read the parquet dataset
    # Grab every .parquet file in the CL folder
    paths = sorted(glob.glob(os.path.join(cl_dir, '*.parquet')))
    # Read & concatenate them into one DataFrame
    df = pd.concat((pd.read_parquet(p) for p in paths), ignore_index=True)

    book = OrderBook()
    for msg in df.itertuples(index=False):
        action = msg.action
        oid = msg.order_id
        # Some messages (e.g. trades) may not have side/price/size
        side = getattr(msg, 'side', None)
        price = getattr(msg, 'price', None)
        size = getattr(msg, 'size', None)

        if action == 'Add':
            book.add_order(oid, side, price, size)
        elif action == 'Modify':
            # here msg.size is the new size
            book.modify_order(oid, size)
        elif action == 'Cancel':
            book.cancel_order(oid)
        elif action in ('Fill', 'Trade'):
            # msg.size is the filled quantity
            book.fill_order(oid, size)
        else:
            # ignore unknown actions
            continue

    return book


if __name__ == '__main__':
    if len(sys.argv) < 2:
        print("Usage: python main.py <path_to_parquet>")
        sys.exit(1)

    parquet_file = sys.argv[1]
    ob = rebuild_orderbook(parquet_file)

    # Example: print top-of-book
    try:
        best_bid = next(iter(ob.bids.items()))
        best_ask = next(iter(ob.asks.items()))
        print(f"Best bid: {best_bid[0]} x {best_bid[1].total_size}")
        print(f"Best ask: {best_ask[0]} x {best_ask[1].total_size}")
    except StopIteration:
        print("Book is empty.")
